In [19]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins

print('Libraries imported.')

Libraries imported.


In [20]:
CLIENT_ID = 'VVDNPACN2HTRYWPH3WKWLSPYKJHLBFMY2QB2EDSC5POCTUSL' # your Foursquare ID
CLIENT_SECRET = 'OVLLVEMZW4SZF5ESVTDBW3FKTW1IW15ZLORCUTAFDM0IXQVO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VVDNPACN2HTRYWPH3WKWLSPYKJHLBFMY2QB2EDSC5POCTUSL
CLIENT_SECRET:OVLLVEMZW4SZF5ESVTDBW3FKTW1IW15ZLORCUTAFDM0IXQVO


In [21]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "los angeles",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [22]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

find coffee shops and associated metadata

In [23]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e2071738130e92fc6a3f821',
  'name': 'Cafe Demitasse',
  'location': {'address': '135 S San Pedro St',
   'crossStreet': 'at 2nd St',
   'lat': 34.04966845839441,
   'lng': -118.24169634169901,
   'labeledLatLngs': [{'label': 'display',
     'lat': 34.04966845839441,
     'lng': -118.24169634169901},
    {'label': 'entrance', 'lat': 34.049761, 'lng': -118.241609}],
   'postalCode': '90012',
   'cc': 'US',
   'city': 'Los Angeles',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['135 S San Pedro St (at 2nd St)',
    'Los Angeles, CA 90012',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.

In [41]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df

{'address': '135 S San Pedro St', 'crossStreet': 'at 2nd St', 'lat': 34.04966845839441, 'lng': -118.24169634169901, 'labeledLatLngs': [{'label': 'display', 'lat': 34.04966845839441, 'lng': -118.24169634169901}, {'label': 'entrance', 'lat': 34.049761, 'lng': -118.241609}], 'postalCode': '90012', 'cc': 'US', 'city': 'Los Angeles', 'state': 'CA', 'country': 'United States', 'formattedAddress': ['135 S San Pedro St (at 2nd St)', 'Los Angeles, CA 90012', 'United States']}
{'address': '116 Judge John Aiso St', 'lat': 34.0509079511909, 'lng': -118.240436223769, 'labeledLatLngs': [{'label': 'display', 'lat': 34.0509079511909, 'lng': -118.240436223769}], 'postalCode': '90012', 'cc': 'US', 'city': 'Los Angeles', 'state': 'CA', 'country': 'United States', 'formattedAddress': ['116 Judge John Aiso St', 'Los Angeles, CA 90012', 'United States']}
{'address': '304 S Broadway', 'lat': 34.050881374258196, 'lng': -118.24795177127038, 'labeledLatLngs': [{'label': 'display', 'lat': 34.050881374258196, 'ln

,uid,name,shortname,address,postalcode,lat,lng
0,4e2071738130e92fc6a3f821,Cafe Demitasse,Coffee Shop,,90012,34.049668,-118.241696
1,51dccd46498e4f9ac4865270,JiST Cafe,Breakfast,,90012,34.050908,-118.240436
2,58680bee9c439d2cc839d367,Blue Bottle Coffee,Coffee Shop,,90013,34.050881,-118.247952
3,4da117b2bf22a143f96894f8,Café Dulcé,Bakery,,90012,34.048869,-118.240508
4,5856f788158b966de017c324,Tilt Coffee,Coffee Shop,,90013,34.048258,-118.246359
5,51a8d473498ea68ee462260c,G&B Coffee,Coffee Shop,,90013,34.051006,-118.249356
6,57fd4578498e20e69bc98c2a,Starbucks,Coffee Shop,,90012,34.049518,-118.241908
7,4b3e9810f964a520789f25e3,Mitsuru Cafe,Japanese,,90012,34.048959,-118.240001
8,4ae21ca4f964a520d58a21e3,Big Man Bakes,Cupcakes,,90013,34.047693,-118.247532
9,5b5a3567c03635002c97f1a1,Bicycle Coffee,Coffee Shop,,90012,34.048184,-118.240115


In [28]:
la_center = d["geocode"]
la_center

{'what': '',
 'where': 'los angeles',
 'center': {'lat': 34.05223, 'lng': -118.24368},
 'displayString': 'Los Angeles, CA, United States',
 'cc': 'US',
 'geometry': {'bounds': {'ne': {'lat': 34.337306, 'lng': -118.155289},
   'sw': {'lat': 33.703652, 'lng': -118.668176}}},
 'slug': 'los-angeles-california',
 'longId': '72057594043296297'}

In [42]:
df

,uid,name,shortname,address,postalcode,lat,lng
0,4e2071738130e92fc6a3f821,Cafe Demitasse,Coffee Shop,,90012,34.049668,-118.241696
1,51dccd46498e4f9ac4865270,JiST Cafe,Breakfast,,90012,34.050908,-118.240436
2,58680bee9c439d2cc839d367,Blue Bottle Coffee,Coffee Shop,,90013,34.050881,-118.247952
3,4da117b2bf22a143f96894f8,Café Dulcé,Bakery,,90012,34.048869,-118.240508
4,5856f788158b966de017c324,Tilt Coffee,Coffee Shop,,90013,34.048258,-118.246359
5,51a8d473498ea68ee462260c,G&B Coffee,Coffee Shop,,90013,34.051006,-118.249356
6,57fd4578498e20e69bc98c2a,Starbucks,Coffee Shop,,90012,34.049518,-118.241908
7,4b3e9810f964a520789f25e3,Mitsuru Cafe,Japanese,,90012,34.048959,-118.240001
8,4ae21ca4f964a520d58a21e3,Big Man Bakes,Cupcakes,,90013,34.047693,-118.247532
9,5b5a3567c03635002c97f1a1,Bicycle Coffee,Coffee Shop,,90012,34.048184,-118.240115


In [40]:
from folium import plugins


map_la = folium.Map(location=[34.05223, -118.24368], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_la)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_la.add_child(plugins.HeatMap(hm_data))

map_la

From this folium map we can see that the downtown area has a high density of coffee shops, leading us to conclude that although this is an area with demand for coffee the existing competitor landscape would be too competitive to break through and that we should investigate other areas beforehand to optimise our market access success.